In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map

### 建立数据样本

In [ ]:
# point = ee.Geometry.Point([-122.4439, 37.7538])
# 建立一个点坐标
point = ee.Geometry.Point([-87.7719, 41.8799])

# 导入landsat8数据
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point) \
    .filterDate('2019-01-01', '2019-12-31') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

In [ ]:
training = image.sample(
    scale= 30,
    numPixels=5000,
    seed= 0,
    geometries=True
)

Map.addLayer(training, {}, 'training', False)
Map

### 建立一个聚类器

In [ ]:
# 分类数
n_clusters = 5
# 创建一个使用k-means聚类方法的聚类器，并以上述得到的样点作为训练样本来训练数据
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

### 图像分类

In [ ]:
# 在图像上使用上述聚类器
result = image.cluster(clusterer)

Map.addLayer(result.randomVisualizer(), {}, 'clusters')
Map

### 修改图像颜色

In [ ]:
legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
legend_colors = ['#00FF00', '#0000FF', '#BEBADA', '#FB8072', '#80B1D3']

result = result.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])

Map.addLayer(result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')
# 建立图例
Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomleft')
Map